In [1]:
import acquire
import os
import pandas as pd
from requests import get
from bs4 import BeautifulSoup


import unicodedata
import re
import json

import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords

import pandas as pd
from acquire import get_blog_articles_data


# acquire from acquire

In [2]:
blog_articles_data = get_blog_articles_data()
blog_articles_data

,title,content
0,Spotlight on APIDA Voices: Celebrating Heritag...,May is traditionally known as Asian American a...
1,Women in tech: Panelist Spotlight – Magdalena ...,Women in tech: Panelist Spotlight – Magdalena ...
2,Women in tech: Panelist Spotlight – Rachel Rob...,Women in tech: Panelist Spotlight – Rachel Rob...
3,Women in Tech: Panelist Spotlight – Sarah Mellor,Women in tech: Panelist Spotlight – Sarah Mell...
4,Women in Tech: Panelist Spotlight – Madeleine ...,Women in tech: Panelist Spotlight – Madeleine ...
5,Black Excellence in Tech: Panelist Spotlight –...,Black excellence in tech: Panelist Spotlight –...


# basic_clean function


In [3]:
article = blog_articles_data

In [5]:
#convert text to all lower case for normalcy
article['content'] = article['content'].str.lower()


In [21]:
# remove accented character, normalize, standardize it into ASCII
article['content'] = article['content'].apply(
    lambda text: unicodedata.normalize('NFKD', text)
    .encode('ascii', 'ignore')
    .decode('utf-8', 'ignore')
)


In [22]:
article['content'[0:100]]

0    may is traditionally known as asian american a...
1    women in tech: panelist spotlight  magdalena r...
2    women in tech: panelist spotlight  rachel robb...
3    women in tech: panelist spotlight  sarah mello...
4    women in tech: panelist spotlight  madeleine c...
5    black excellence in tech: panelist spotlight  ...
Name: content, dtype: object

In [26]:
# removing special characters
import re

article['content'] = article['content'].apply(
    lambda text: re.sub(r"[^a-z0-9'\s]", '', text)
)


In [27]:
article['content']

0    may is traditionally known as asian american a...
1    women in tech panelist spotlight  magdalena ra...
2    women in tech panelist spotlight  rachel robbi...
3    women in tech panelist spotlight  sarah mellor...
4    women in tech panelist spotlight  madeleine ca...
5    black excellence in tech panelist spotlight  w...
Name: content, dtype: object

# tokenize function

In [29]:
tokenizer = nltk.tokenize.ToktokTokenizer()
tokenizer.tokenize(article['content'], return_str=True)[0:200]

'0 may is traditionally known as asian american a ... \n1 women in tech panelist spotlight magdalena ra ... \n2 women in tech panelist spotlight rachel robbi ... \n3 women in tech panelist spotlight sarah'

# Stem function

In [47]:
import nltk
from nltk.stem import PorterStemmer
import pandas as pd

# Download the 'punkt' dataset if you haven't already
nltk.download('punkt')

ps = PorterStemmer()

# Define a function to split and stem the text
def stem_text(text):
    words = nltk.word_tokenize(text)  # Tokenize the text
    stemmed_words = [ps.stem(word) for word in words]
    return ' '.join(stemmed_words)

# Assuming 'article' is a DataFrame and 'content' is the column to be stemmed
article['content'] = article['content'].apply(stem_text)

# Now, you can count and display the most common stemmed words
stemmed_words = ' '.join(article['content']).split()
word_counts = pd.Series(stemmed_words).value_counts().head(10)
print(word_counts)


in        79
to        77
the       64
and       59
a         53
of        38
as        30
is        29
on        28
codeup    27
dtype: int64


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/lordvoldemort/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Lemmatize function

In [60]:
import nltk
from nltk.stem import WordNetLemmatizer

# Download the 'punkt' and 'wordnet' datasets if you haven't already
nltk.download('punkt')
nltk.download('wordnet')  # Download the WordNet dataset for lemmatization

# Create a WordNet lemmatizer object
wnl = WordNetLemmatizer()

# Define a function to split and lemmatize the text
def lemmatize_text(text):
    words = nltk.word_tokenize(text)  # Tokenize the text
    lemmatized_words = [wnl.lemmatize(word) for word in words]
    return ' '.join(lemmatized_words)

# Assuming 'article' is a DataFrame and 'content' is the column to be lemmatized
article['content'] = article['content'].apply(lemmatize_text)

# Now, you can count and display the most common lemmatized words
lemmatized_words = ' '.join(article['content']).split()
word_counts = pd.Series(lemmatized_words).value_counts().head(10)
print(word_counts)


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/lordvoldemort/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/lordvoldemort/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


LookupError: 
**********************************************************************
  Resource [93momw-1.4[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('omw-1.4')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/omw-1.4[0m

  Searched in:
    - '/Users/lordvoldemort/nltk_data'
    - '/opt/homebrew/anaconda3/nltk_data'
    - '/opt/homebrew/anaconda3/share/nltk_data'
    - '/opt/homebrew/anaconda3/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


# remove_stopwords function

In [99]:
from nltk.corpus import stopwords

# Assuming you have 'article' DataFrame and 'content' column
stopword_list = stopwords.words('english')

# Remove stopwords from the 'content' column
article['content'] = article['content'].apply(
    lambda text: ' '.join([word for word in text.split() if word.lower() not in stopword_list])
)

# Print a portion of the article with stopwords removed
print(article['content'][0:100])

# Count the total number of stopwords removed
removed_stopwords_count = sum(article['content'].str.split().apply(lambda words: len([word for word in words if word.lower() in stopword_list])))
print('Removed {} stopwords'.format(removed_stopwords_count))


0    may tradit known asian american pacif island a...
1    women tech panelist spotlight magdalena rahn c...
2    women tech panelist spotlight rachel robbinsma...
3    women tech panelist spotlight sarah mellor cod...
4    women tech panelist spotlight madelein capper ...
5    black excel tech panelist spotlight wilmari de...
Name: content, dtype: object
Removed 0 stopwords


# new_df function

In [101]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

# Download the 'stopwords' dataset if you haven't already
nltk.download('stopwords')

# Initialize the Porter Stemmer and stopwords list
ps = PorterStemmer()
stopword_list = set(stopwords.words('english'))

# Define a function to preprocess text
def preprocess_text(text):
    # Normalize text (e.g., lowercase)
    text = text.lower()
    # Tokenize the text
    words = nltk.word_tokenize(text)
    # Remove stopwords and apply stemming
    words = [ps.stem(word) for word in words if word not in stopword_list]
    return ' '.join(words)

# Create new columns in the DataFrame
article['original'] = article['content']
article['clean'] = article['content'].apply(preprocess_text)
article['stemmed'] = article['clean'].apply(lambda text: ' '.join([ps.stem(word) for word in text.split()]))

# You already have the 'title' column in your DataFrame

# Display the resulting DataFrame
article[['title', 'original', 'clean', 'stemmed']]


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/lordvoldemort/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,title,original,clean,stemmed
0,Spotlight on APIDA Voices: Celebrating Heritag...,may tradit known asian american pacif island a...,may tradit known asian american pacif island a...,may tradit known asian american pacif island a...
1,Women in tech: Panelist Spotlight – Magdalena ...,women tech panelist spotlight magdalena rahn c...,women tech panelist spotlight magdalena rahn c...,women tech panelist spotlight magdalena rahn c...
2,Women in tech: Panelist Spotlight – Rachel Rob...,women tech panelist spotlight rachel robbinsma...,women tech panelist spotlight rachel robbinsma...,women tech panelist spotlight rachel robbinsma...
3,Women in Tech: Panelist Spotlight – Sarah Mellor,women tech panelist spotlight sarah mellor cod...,women tech panelist spotlight sarah mellor cod...,women tech panelist spotlight sarah mellor cod...
4,Women in Tech: Panelist Spotlight – Madeleine ...,women tech panelist spotlight madelein capper ...,women tech panelist spotlight madelein capper ...,women tech panelist spotlight madelein capper ...
5,Black Excellence in Tech: Panelist Spotlight –...,black excel tech panelist spotlight wilmari de...,black excel tech panelist spotlight wilmari de...,black excel tech panelist spotlight wilmari de...


# code_df data frame